# 🪙 Examination of features using LGBM
---

The objective of this notebook is to explore effective features using the importance of LGBM features..<br>
First, training by LGBM. Then check the importance of each feature.<br>
このノートブックでは、LGBMのFeature Importanceを可視化して、効果的な特徴量を探索しています。
まず、LGBMで学習をし、その後、Feature Importanceを確認します。
---
![](https://storage.googleapis.com/kaggle-competitions/kaggle/30894/logos/header.png)
---
References: <br>
1. [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)<br>
2. [🪙💲 G-Research- Starter LGBM Pipeline](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline)<br>
3. [Feature Engineering: Rolling Aggregations](https://www.kaggle.com/yamqwe/feature-engineering-rolling-aggregations?scriptVersionId=81111556)<br>

Please if this kernel is useful, please upvote !! (宜しければUpvoteお願いします！)

# Libraries

In [ ]:
import gresearch_crypto

import os
from tqdm import tqdm
import traceback
import random
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
plt.rcParams.update({'font.size': 18})
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 150)

# Config

In [ ]:
DEVICE = "GPU" #or "TPU"

SEED = 42

# CV PARAMS
FOLDS = 2
GROUP_GAP = 130
MAX_TEST_GROUP_SIZE = 180
MAX_TRAIN_GROUP_SIZE = 720

# LOAD STRICT? YES=1 NO=0 | see: https://www.kaggle.com/julian3833/proposal-for-a-meaningful-lb-strict-lgbm
LOAD_STRICT = True

# WHICH YEARS TO INCLUDE? YES=1 NO=0
INC2021 = 0
INC2020 = 0
INC2019 = 0
INC2018 = 0
INC2017 = 0
INCCOMP = 1
INCSUPP = 0

# BATCH SIZE AND EPOCHS (for CNN)
# BATCH_SIZES = [2048] * FOLDS # [2048, 2048, 2048, 2048, 2048]
# EPOCHS = [1] * FOLDS         # [1, 1, 1, 1, 1]

# FEAT_WIN_SIZE = 10 # Window size for feature extraction

In [ ]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

fix_all_seeds(SEED)

# Data Loading

In [ ]:
import datatable as dt
extra_data_files = {0: '../input/cryptocurrency-extra-data-binance-coin', 
                    1: '../input/cryptocurrency-extra-data-bitcoin', 
                    2: '../input/cryptocurrency-extra-data-bitcoin-cash', 
                    3: '../input/cryptocurrency-extra-data-cardano', 
                    4: '../input/cryptocurrency-extra-data-dogecoin', 
                    5: '../input/cryptocurrency-extra-data-eos-io', 
                    6: '../input/cryptocurrency-extra-data-ethereum', 
                    7: '../input/cryptocurrency-extra-data-ethereum-classic', 
                    8: '../input/cryptocurrency-extra-data-iota', 
                    9: '../input/cryptocurrency-extra-data-litecoin', 
                    11: '../input/cryptocurrency-extra-data-monero', 
                    10: '../input/cryptocurrency-extra-data-maker', 
                    12: '../input/cryptocurrency-extra-data-stellar', 
                    13: '../input/cryptocurrency-extra-data-tron'
                    }

# Uncomment to load the original csv [slower]
# orig_df_train = pd.read_csv(data_path + 'train.csv') 
# supp_df_train = pd.read_csv(data_path + 'supplemental_train.csv')
# df_asset_details = pd.read_csv(data_path  + 'asset_details.csv').sort_values("Asset_ID")

orig_df_train = dt.fread('../input/cryptocurrency-extra-data-binance-coin/orig_train.jay').to_pandas()
df_asset_details = dt.fread('../input/cryptocurrency-extra-data-binance-coin/orig_asset_details.jay').to_pandas()
supp_df_train = dt.fread('../input/cryptocurrency-extra-data-binance-coin/orig_supplemental_train.jay').to_pandas()
assets_details = dt.fread('../input/cryptocurrency-extra-data-binance-coin/orig_asset_details.jay').to_pandas()
asset_weight_dict = {assets_details['Asset_ID'].tolist()[idx]: assets_details['Weight'].tolist()[idx] for idx in range(len(assets_details))}
asset_name_dict = {assets_details['Asset_ID'].tolist()[idx]: assets_details['Asset_Name'].tolist()[idx] for idx in range(len(assets_details))}

In [ ]:
def load_training_data_for_asset(asset_id, load_jay = True):
    """Asset_idごとのトレーニングデータを生成する。

    Args:
        asset_id (int): アセットID
        load_jay (bool, optional): jayデータをロードするか. Defaults to True.

    Returns:
        DataFrame: AssetIdのデータフレーム
    """
    dfs = []
    
    # オリジナルから、AssetIDの該当するデータを読み込む。
    if INCCOMP: 
        dfs.append(orig_df_train[orig_df_train["Asset_ID"] == asset_id].copy())
    if INCSUPP: 
        dfs.append(supp_df_train[supp_df_train["Asset_ID"] == asset_id].copy())
    
    if load_jay:
        if INC2017 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2017) + '.csv'): 
            dfs.append(dt.fread(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2017) + '.jay').to_pandas())
        if INC2018 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2018) + '.csv'): 
            dfs.append(dt.fread(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2018) + '.jay').to_pandas())
        if INC2019 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2019) + '.csv'): 
            dfs.append(dt.fread(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2019) + '.jay').to_pandas())
        if INC2020 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2020) + '.csv'): 
            dfs.append(dt.fread(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2020) + '.jay').to_pandas())
        if INC2021 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2021) + '.csv'): 
            dfs.append(dt.fread(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2021) + '.jay').to_pandas())
    else:
        # CSVで読み込む
        if INC2017 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2017) + '.csv'): 
            dfs.append(pd.read_csv(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2017) + '.csv'))
        if INC2018 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2018) + '.csv'): 
            dfs.append(pd.read_csv(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2018) + '.csv'))
        if INC2019 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2019) + '.csv'): 
            dfs.append(pd.read_csv(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2019) + '.csv'))
        if INC2020 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2020) + '.csv'): 
            dfs.append(pd.read_csv(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2020) + '.csv'))
        if INC2021 and os.path.exists(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2021) + '.csv'): 
            dfs.append(pd.read_csv(extra_data_files[asset_id] + '/full_data__' + str(asset_id) + '__' + str(2021) + '.csv'))
    
    # dfsリストに格納していたDFをCONCAT        
    df = pd.concat(dfs, axis = 0) if len(dfs) > 1 else dfs[0]
    df['date'] = pd.to_datetime(df['timestamp'], unit = 's')
    if LOAD_STRICT: 
        df = df.loc[df['date'] < "2021-06-13 00:00:00"]
    df = df.sort_values('date')
    return df

In [ ]:
def load_data_for_all_assets():
    """各AssetIDのデータフレームをload_training_data_for_assetでロードした後に
        CONCATし、1つのデータフレームにする。

    Returns:
        DataFrame: 全AssetIDを含むDATAFRAME
    """
    dfs = []
    for asset_id in list(extra_data_files.keys()): 
        dfs.append(load_training_data_for_asset(asset_id))
    return pd.concat(dfs)

# Feature Engineering

In [ ]:
#####################################################################
# base stats
#####################################################################
def mean(x):
    return np.mean(x)
def length(x):
    return len(x)
def standard_deviation(x):
    return np.std(x)

def variation_coefficient(x):
    mean = np.mean(x)
    if mean != 0:
        return np.std(x) / mean
    else:
        return np.nan

def variance(x):
    return np.var(x)

def skewness(x):
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.skew(x)

def kurtosis(x):
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.kurtosis(x)

#####################################################################
# higher_order_stats
#####################################################################
def abs_energy(x):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.dot(x, x)

def root_mean_square(x):
    return np.sqrt(np.mean(np.square(x))) if len(x) > 0 else np.NaN

def sum_values(x):
    if len(x) == 0:
        return 0
    return np.sum(x)

# def realized_volatility(series_log_return):
#     return np.sqrt(np.sum(series_log_return**2))
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

def realized_abs_skew(series):
    return np.power(np.abs(np.sum(series**3)),1/3)

def realized_skew(series):
    return np.sign(np.sum(series**3))*np.power(np.abs(np.sum(series**3)),1/3)

def realized_vol_skew(series):
    return np.power(np.abs(np.sum(series**6)),1/6)

def realized_quarticity(series):
    return np.power(np.sum(series**4),1/4)
#####################################################################
# min_median_max
#####################################################################
def minimum(x):
    return np.min(x)
def median(x):
    return np.median(x)
def maximum(x):
    return np.max(x)
#####################################################################
# additional_quantile
#####################################################################
quantile_01 = lambda x: quantile(x,0.1)
quantile_01.__name__ = 'quantile_01'

quantile_025 = lambda x: quantile(x,0.25)
quantile_025.__name__ = 'quantile_025'

quantile_075 = lambda x: quantile(x,0.75)
quantile_075.__name__ = 'quantile_075'

quantile_09 = lambda x: quantile(x,0.9)
quantile_09.__name__ = 'quantile_09'
#####################################################################
# other_minmax
#####################################################################
def absolute_maximum(x):
    return np.max(np.absolute(x)) if len(x) > 0 else np.NaN

def max_over_min(series):
    if len(series)<2:
        return 0
    if np.min(series) == 0:
        return np.nan
    return np.max(series)/np.min(series)

def max_over_min_sq(series):
    if len(series)<2:
        return 0
    if np.min(series) == 0:
        return np.nan
    return np.square(np.max(series)/np.min(series))

#####################################################################
# minmax_positions
#####################################################################
def last_location_of_maximum(x):
    x = np.asarray(x)
    return 1.0 - np.argmax(x[::-1]) / len(x) if len(x) > 0 else np.NaN

def first_location_of_maximum(x):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.argmax(x) / len(x) if len(x) > 0 else np.NaN

def last_location_of_minimum(x):
    x = np.asarray(x)
    return 1.0 - np.argmin(x[::-1]) / len(x) if len(x) > 0 else np.NaN

def first_location_of_minimum(x):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.argmin(x) / len(x) if len(x) > 0 else np.NaN

#####################################################################
# peaks
#####################################################################
number_peaks_2 = lambda x: number_peaks(x,2)
number_peaks_2.__name__ = 'number_peaks_2'

mean_n_absolute_max_2 = lambda x: mean_n_absolute_max(x,2)
mean_n_absolute_max_2.__name__ = 'mean_n_absolute_max_2'

number_peaks_5 = lambda x: number_peaks(x,5)
number_peaks_5.__name__ = 'number_peaks_5'

mean_n_absolute_max_5 = lambda x: mean_n_absolute_max(x,5)
mean_n_absolute_max_5.__name__ = 'mean_n_absolute_max_5'

number_peaks_10 = lambda x: number_peaks(x,10)
number_peaks_10.__name__ = 'number_peaks_10'

mean_n_absolute_max_10 = lambda x: mean_n_absolute_max(x,10)
mean_n_absolute_max_10.__name__ = 'mean_n_absolute_max_10'

#####################################################################
# counts
#####################################################################
def count_unique(series):
    return len(np.unique(series))

def count(series):
    return series.size

count_above_0 = lambda x: count_above(x,0)
count_above_0.__name__ = 'count_above_0'

count_below_0 = lambda x: count_below(x,0)
count_below_0.__name__ = 'count_below_0'

value_count_0 = lambda x: value_count(x,0)
value_count_0.__name__ = 'value_count_0'

count_near_0 = lambda x: range_count(x,-0.00001,0.00001)
count_near_0.__name__ = 'count_near_0_0'

#####################################################################
# reoccurring values
#####################################################################
def count_above_mean(x):
    m = np.mean(x)
    return np.where(x > m)[0].size

def count_below_mean(x):
    m = np.mean(x)
    return np.where(x < m)[0].size

# Test non-consecutive non-reoccuring values ?
def percentage_of_reoccurring_values_to_all_values(x):
    if len(x) == 0:
        return np.nan
    unique, counts = np.unique(x, return_counts=True)
    if counts.shape[0] == 0:
        return 0
    return np.sum(counts > 1) / float(counts.shape[0])

def percentage_of_reoccurring_datapoints_to_all_datapoints(x):
    if len(x) == 0:
        return np.nan
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    value_counts = x.value_counts()
    reoccuring_values = value_counts[value_counts > 1].sum()
    if np.isnan(reoccuring_values):
        return 0

    return reoccuring_values / x.size

def sum_of_reoccurring_values(x):
    unique, counts = np.unique(x, return_counts=True)
    counts[counts < 2] = 0
    counts[counts > 1] = 1
    return np.sum(counts * unique)

def sum_of_reoccurring_data_points(x):
    unique, counts = np.unique(x, return_counts=True)
    counts[counts < 2] = 0
    return np.sum(counts * unique)

def ratio_value_number_to_time_series_length(x):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    if x.size == 0:
        return np.nan

    return np.unique(x).size / x.size

#####################################################################
# count suplicates
#####################################################################
def count_duplicate_max(x):
    return np.sum(x == np.max(x))

def count_duplicate_min(x):
    return np.sum(x == np.min(x))

def count_duplicate(x):
    return x.size - np.unique(x).size

#####################################################################
# valiations
#####################################################################
def mean_abs_change(x):
    return np.mean(np.abs(np.diff(x)))

def mean_change(x):
    x = np.asarray(x)
    return (x[-1] - x[0]) / (len(x) - 1) if len(x) > 1 else np.NaN

def mean_second_derivative_central(x):
    x = np.asarray(x)
    return (x[-1] - x[-2] - x[1] + x[0]) / (2 * (len(x) - 2)) if len(x) > 2 else np.NaN

def absolute_sum_of_changes(x):
    return np.sum(np.abs(np.diff(x)))

number_crossing_0 = lambda x: number_crossing_m(x,0)
number_crossing_0.__name__ = 'number_crossing_0'

#####################################################################
# ranges
#####################################################################
def variance_std_ratio(x):
    y = np.var(x)
    if y != 0:
        return y/np.sqrt(y)
    else:
        return np.nan

def ratio_beyond_r_sigma(x, r):
    if x.size == 0:
        return np.nan
    else:
        return np.sum(np.abs(x - np.mean(x)) > r * np.asarray(np.std(x))) / x.size

ratio_beyond_01_sigma = lambda x: ratio_beyond_r_sigma(x,0.1)
ratio_beyond_01_sigma.__name__ = 'ratio_beyond_01_sigma'

ratio_beyond_02_sigma = lambda x: ratio_beyond_r_sigma(x,0.2)
ratio_beyond_02_sigma.__name__ = 'ratio_beyond_02_sigma'

ratio_beyond_03_sigma = lambda x: ratio_beyond_r_sigma(x,0.3)
ratio_beyond_03_sigma.__name__ = 'ratio_beyond_03_sigma'

def large_standard_deviation(x):
    if (np.max(x)-np.min(x)) == 0:
        return np.nan
    else:
        return np.std(x)/(np.max(x)-np.min(x))
    
def range_ratio(x):
    mean_median_difference = np.abs(np.mean(x) - np.median(x))
    max_min_difference = np.max(x) - np.min(x)
    if max_min_difference == 0:
        return np.nan
    else:
        return mean_median_difference / max_min_difference
    
#####################################################################
# get first fn
#####################################################################
get_first = lambda x: x.iloc[0]
get_first.__name__ = 'get_first'

get_last = lambda x: x.iloc[-1]
get_last.__name__ = 'get_last'

#####################################################################
# draw functions
#####################################################################
#drawdons functions are mine
def maximum_drawdown(series):
    series = np.asarray(series)
    if len(series)<2:
        return 0
    k = series[np.argmax(np.maximum.accumulate(series) - series)]
    i = np.argmax(np.maximum.accumulate(series) - series)
    if len(series[:i])<1:
        return np.NaN
    else:
        j = np.max(series[:i])
    return j-k

def maximum_drawup(series):
    series = np.asarray(series)
    if len(series)<2:
        return 0
    
    series = - series
    k = series[np.argmax(np.maximum.accumulate(series) - series)]
    i = np.argmax(np.maximum.accumulate(series) - series)
    if len(series[:i])<1:
        return np.NaN
    else:
        j = np.max(series[:i])
    return j-k

def drawdown_duration(series):
    series = np.asarray(series)
    if len(series)<2:
        return 0

    k = np.argmax(np.maximum.accumulate(series) - series)
    i = np.argmax(np.maximum.accumulate(series) - series)
    if len(series[:i]) == 0:
        j=k
    else:
        j = np.argmax(series[:i])
    return k-j

def drawup_duration(series):
    series = np.asarray(series)
    if len(series)<2:
        return 0

    series=-series
    k = np.argmax(np.maximum.accumulate(series) - series)
    i = np.argmax(np.maximum.accumulate(series) - series)
    if len(series[:i]) == 0:
        j=k
    else:
        j = np.argmax(series[:i])
    return k-j

#####################################################################
# Others
#####################################################################
def _roll(a, shift):
    """ Roll 1D array elements. Improves the performance of numpy.roll()"""

    if not isinstance(a, np.ndarray):
        # 型が合っていなかったら直す。
        a = np.asarray(a)
    idx = shift % len(a)
    return np.concatenate([a[-idx:], a[:-idx]])

def _get_length_sequences_where(x):
    """ This method calculates the length of all sub-sequences where the array x is either True or 1. """
    if len(x) == 0:
        return [0]
    else:
        res = [len(list(group)) for value, group in itertools.groupby(x) if value == 1]
        return res if len(res) > 0 else [0]

def _aggregate_on_chunks(x, f_agg, chunk_len):
    """Takes the time series x and constructs a lower sampled version of it by applying the aggregation function f_agg on
    consecutive chunks of length chunk_len"""
    
    return [
        getattr(x[i * chunk_len : (i + 1) * chunk_len], f_agg)()
        for i in range(int(np.ceil(len(x) / chunk_len)))
    ]

def _into_subchunks(x, subchunk_length, every_n=1):
    """Split the time series x into subwindows of length "subchunk_length", starting every "every_n"."""
    len_x = len(x)

    assert subchunk_length > 1
    assert every_n > 0

    # how often can we shift a window of size subchunk_length over the input?
    num_shifts = (len_x - subchunk_length) // every_n + 1
    shift_starts = every_n * np.arange(num_shifts)
    indices = np.arange(subchunk_length)

    indexer = np.expand_dims(indices, axis=0) + np.expand_dims(shift_starts, axis=1)
    return np.asarray(x)[indexer]

def set_property(key, value):
    """
    This method returns a decorator that sets the property key of the function to value
    """

    def decorate_func(func):
        setattr(func, key, value)
        if func.__doc__ and key == "fctype":
            func.__doc__ = (
                func.__doc__ + "\n\n    *This function is of type: " + value + "*\n"
            )
        return func

    return decorate_func

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

def mean(x):
    return np.mean(x)

def length(x):
    return len(x)

def standard_deviation(x):
    return np.std(x)

def variation_coefficient(x):
    mean = np.mean(x)
    if mean != 0:
        return np.std(x) / mean
    else:
        return np.nan
    
def has_duplicate_max(x):
    return np.sum(x == np.max(x)) >= 2

def has_duplicate_min(x):
    return np.sum(x == np.min(x)) >= 2

def has_duplicate(x):
    return x.size != np.unique(x).size

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def mean_n_absolute_max(x, number_of_maxima = 1):
    """ Calculates the arithmetic mean of the n absolute maximum values of the time series."""
    assert (
        number_of_maxima > 0
    ), f" number_of_maxima={number_of_maxima} which is not greater than 1"

    n_absolute_maximum_values = np.sort(np.absolute(x))[-number_of_maxima:]
    return np.mean(n_absolute_maximum_values) if len(x) > number_of_maxima else np.NaN

def count_above(x, t):
    if len(x)==0:
        return np.nan
    else:
        return np.sum(x >= t) / len(x)

def count_below(x, t):
    if len(x)==0:
        return np.nan
    else:
        return np.sum(x <= t) / len(x)

#number of valleys = number_peaks(-x, n)
def number_peaks(x, n):
    """
    Calculates the number of peaks of at least support n in the time series x. A peak of support n is defined as a
    subsequence of x where a value occurs, which is bigger than its n neighbours to the left and to the right.
    """
    x_reduced = x[n:-n]

    res = None
    for i in range(1, n + 1):
        result_first = x_reduced > _roll(x, i)[n:-n]

        if res is None:
            res = result_first
        else:
            res &= result_first

        res &= x_reduced > _roll(x, -i)[n:-n]
    return np.sum(res)

def longest_strike_below_mean(x):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.max(_get_length_sequences_where(x < np.mean(x))) if x.size > 0 else 0

def longest_strike_above_mean(x):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.max(_get_length_sequences_where(x > np.mean(x))) if x.size > 0 else 0

def quantile(x, q):
    if len(x) == 0:
        return np.NaN
    return np.quantile(x, q)

# crossing the mean ? other levels ? 
def number_crossing_m(x, m):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    # From https://stackoverflow.com/questions/3843017/efficiently-detect-sign-changes-in-python
    positive = x > m
    return np.where(np.diff(positive))[0].size

def value_count(x, value):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    if np.isnan(value):
        return np.isnan(x).sum()
    else:
        return x[x == value].size

def range_count(x, min, max):
    return np.sum((x >= min) & (x < max))

In [ ]:
# Difine methods list
base_stats = [mean,sum,length,standard_deviation,variation_coefficient,variance,skewness,kurtosis]
higher_order_stats = [abs_energy,root_mean_square,sum_values,realized_volatility,realized_abs_skew,realized_skew,realized_vol_skew,realized_quarticity]
min_median_max = [minimum,median,maximum]
additional_quantiles = [quantile_01,quantile_025,quantile_075,quantile_09]
other_min_max = [absolute_maximum,max_over_min,max_over_min_sq]
min_max_positions = [last_location_of_maximum,first_location_of_maximum,last_location_of_minimum,first_location_of_minimum]
peaks = [number_peaks_2, mean_n_absolute_max_2, number_peaks_5, mean_n_absolute_max_5, number_peaks_10, mean_n_absolute_max_10]
counts = [count_unique,count,count_above_0,count_below_0,value_count_0,count_near_0]
reoccuring_values = [count_above_mean,count_below_mean,percentage_of_reoccurring_values_to_all_values,percentage_of_reoccurring_datapoints_to_all_datapoints,sum_of_reoccurring_values,sum_of_reoccurring_data_points,ratio_value_number_to_time_series_length]
count_duplicates = [count_duplicate,count_duplicate_min,count_duplicate_max]
variations = [mean_abs_change,mean_change,mean_second_derivative_central,absolute_sum_of_changes,number_crossing_0]
ranges = [variance_std_ratio,ratio_beyond_01_sigma,ratio_beyond_02_sigma,ratio_beyond_03_sigma,large_standard_deviation,range_ratio]
get_first_fn = [get_first,get_last]
draw_functions = [maximum_drawdown,maximum_drawup,drawdown_duration,drawup_duration]

In [ ]:
all_agg_functions_ever =\
    base_stats \
    + higher_order_stats \
    + min_median_max \
    + additional_quantiles \
    + other_min_max \
    + min_max_positions \
    + peaks + counts \
    + variations + ranges \
    + count_duplicates \
    + reoccuring_values \
    + get_first_fn \
    + draw_functions
for i in all_agg_functions_ever: print(i.__name__)

In [ ]:
# TODO: Try different features here!
# This is for you to play around, add or remove lists here to extract different featutres out of different sequences
baseline_feature_dict = {
    'Open':           min_median_max,
    'High':           [],
    'Low':            [],
    'Close':          base_stats + higher_order_stats,
    'Count':          [],
    'Volume':         [],
    'VWAP':           [],
}

create_feature_dict = baseline_feature_dict

In [ ]:
# technical indicators
def RSI(close: pd.DataFrame, period: int = 14) -> pd.Series:
    # https://gist.github.com/jmoz/1f93b264650376131ed65875782df386
    """See source https://github.com/peerchemist/finta
    and fix https://www.tradingview.com/wiki/Talk:Relative_Strength_Index_(RSI)
    Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements.
    RSI oscillates between zero and 100. Traditionally, and according to Wilder, RSI is considered overbought when above 70 and oversold when below 30.
    Signals can also be generated by looking for divergences, failure swings and centerline crossovers.
    RSI can also be used to identify the general trend."""

    delta = close.diff()

    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    _gain = up.ewm(com=(period - 1), min_periods=period).mean()
    _loss = down.abs().ewm(com=(period - 1), min_periods=period).mean()

    RS = _gain / _loss
    return pd.Series(100 - (100 / (1 + RS)))

def EMA1(x, n):
    """
    https://qiita.com/MuAuan/items/b08616a841be25d29817
    """
    a= 2/(n+1)
    return pd.Series(x).ewm(alpha=a).mean()

def MACD(close : pd.DataFrame, span1=12, span2=26, span3=9):
    """
    Compute MACD
    # https://www.learnpythonwithrune.org/pandas-calculate-the-moving-average-convergence-divergence-macd-for-a-stock/
    """
    exp1 = EMA1(close, span1)
    exp2 = EMA1(close, span2)
    macd = 100 * (exp1 - exp2) / exp2
    signal = EMA1(macd, span3)

    return macd, signal

In [ ]:
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

def hlco_ratio(df):
    return (df['High']-df['Low']) / (df['Close']-df['Open'])

In [ ]:
def get_features(df,row=False):    
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Asset_ID', 'date']].copy()
    ## Adding some more features(2021/01/04)
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    # df_feat['hlco_ratio'] = hlco_ratio(df_feat)
    
    df_feat["Close/Open"] = df_feat["Close"] / df_feat["Open"] 
    df_feat["Close-Open"] = df_feat["Close"] - df_feat["Open"] 
    df_feat["High-Low"] = df_feat["High"] - df_feat["Low"] 
    df_feat["High/Low"] = df_feat["High"] / df_feat["Low"]
    
    if row:
        df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean()
    else:
        df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    df_feat["High/Mean"] = df_feat["High"] / df_feat["Mean"]
    df_feat["Low/Mean"] = df_feat["Low"] / df_feat["Mean"]
    df_feat["Volume/Count"] = df_feat["Volume"] / (df_feat["Count"] + 1)
  
    times = pd.to_datetime(df["timestamp"],unit="s",infer_datetime_format=True)
    if row:
        df_feat["hour"] = times.hour  # .dt
        df_feat["dayofweek"] = times.dayofweek 
        df_feat["day"] = times.day 
    else:
        df_feat["hour"] = times.dt.hour  # .dt
        df_feat["dayofweek"] = times.dt.dayofweek 
        df_feat["day"] = times.dt.day       
        
    if row:
        df_feat["Median"] = df_feat[["Open", "High", "Low", "Close"]].median()
    else:
        df_feat["Median"] = df_feat[["Open", "High", "Low", "Close"]].median(axis=1)
    df_feat["High/Median"] = df_feat["High"] / df_feat["Median"]
    df_feat["Low/Median"] = df_feat["Low"] / df_feat["Median"]
    
    for col in ['Open', 'High', 'Low', 'Close', 'VWAP']:
        df_feat[f"Log_1p_{col}"] = np.log1p(df_feat[col])
        
    # RSI
    #df_feat["RSI"] = RSI(df_feat["Close"], 14)

    # MACD
    macd, macd_signal = MACD(df_feat["Close"], 12, 26, 9) 
    df_feat["MACD"] = macd
    df_feat["MACD_signal"] = macd_signal
    
    ## (2021/01/04)
    
    try:
        df_feat['window'] = df_feat['date'].dt.day.astype(str) # date列の日をwindowに入れる。
    except:
        df_feat['window'] = str(df_feat['date'].day)
        
    df_feat.drop(columns = 'date', inplace = True)        
    grp = df_feat.groupby('window').agg(create_feature_dict) # FeatureEng 
    df_feat = df_feat.set_index('window').merge(grp, left_index=True, right_index=True).reset_index()
    df_feat.columns = ['_'.join(col) if type(col) is not str else col for col in df_feat.columns]    
    if 'window' in df_feat.columns: df_feat.drop(columns = 'window', inplace = True)    
    return df_feat

# Configure the model

In [ ]:
# Numpy Version
def corr(a, b, w):
    cov = lambda x, y: np.sum(w * (x - np.average(x, weights=w)) * (y - np.average(y, weights=w))) / np.sum(w)
    return cov(a, b) / np.sqrt(cov(a, a) * cov(b, b))

# TF Version
def tf_cov(x, y, w): 
    return (tf.reduce_sum(w * (x - tf.reduce_mean(x * w)) * (y - tf.reduce_mean(y * w))) / tf.reduce_sum(w))
def tf_comp_metric(a, b, w): 
    return tf_cov(a, b, w) / tf.sqrt(tf_cov(a, a, w) * tf_cov(b, b, w))
def nn_comp_metric(w): 
    def wcorr(x, y): 
        return tf_comp_metric(x, y ,w)
    return wcorr

# Training

### Time Series Cross Validation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]

                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size

            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            if self.verbose > 0:
                    pass

            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]

                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size

            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            if self.verbose > 0:
                    pass

            yield [int(i) for i in train_array], [int(i) for i in test_array]

### Main Training Function

In [ ]:
def replace_outlier(df):
    #四分位数
    q1 = df.quantile(.01)
    q3 = df.quantile(.99)
    
    #外れ値をクリップする
    df = df.clip(q1, q3, axis=1)
    return df

In [ ]:
######################
# Parameters of LGBM
######################
best_params = {
    "objective": "regression",
    "n_estimators" : 637,     # <-- (9) change from 200 to 500
    "boosting_type" : "gbdt",
    "max_depth": -1,
    "num_leaves" : 800,       # <-- (10) Added parameter
    "learning_rate" : 0.092534,   # <-- (10) Added parameter
    "colsample_bytree":0.738963,
    "random_seed" : 2022,
    "lambda_l1" : 1,
    "lambda_l2" : 1, 
    # "tree_method" : 'gpu_hist',  # THE MAGICAL PARAMETER
    }

best_params = {
    'n_estimators':1500,  # 1500,
    'num_leaves':700,
    'objective':"regression",
    'metric':"rmse",
    'boosting_type':"gbdt",
    'learning_rate':0.01,
    'random_state':71,
    'verbose':1,
    'force_col_wise':True,
    }

In [ ]:
from sklearn.preprocessing import RobustScaler
# USE VERBOSE=0 for silent, VERBOSE=1 for interactive, VERBOSE=2 for commit
VERBOSE = 2
global features

def get_Xy_and_model_for_asset(asset_id):
    df = load_training_data_for_asset(asset_id)
    df.reset_index(drop=True, inplace=True) # Fixed220104
    df_proc = get_features(df)
    df_proc['date'] = df['date'].copy()
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    groups = pd.factorize(X['date'].dt.day.astype(str) + '_' + X['date'].dt.month.astype(str) + '_' + X['date'].dt.year.astype(str))[0]
    X = X.drop(columns = 'date')
    # Clipping (add 220104)
    X = replace_outlier(X)
    # Scaler (add 220104)
    scaler = RobustScaler().fit(X)
    tmp_X = scaler.transform(X)
    X = pd.DataFrame(tmp_X, columns=X.columns)
    oof_preds = np.zeros(len(X))
    
    scores, models = [], []
    for fold, (train_idx, val_idx) in enumerate(PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap = GROUP_GAP, max_train_group_size = MAX_TRAIN_GROUP_SIZE, max_test_group_size = MAX_TEST_GROUP_SIZE).split(X, y, groups)):
        # GET TRAINING, VALIDATION SET
        x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        global features
        features = list(X.columns)
        
        # DISPLAY FOLD INFO
        # if DEVICE == 'TPU':
        #     if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
        print('#'*25)
        print('#### FOLD',fold+1)       
        
        # BUILD MODEL and TRAIN
        model = LGBMRegressor(**best_params)       
        model.fit(x_train, y_train)     

        # PREDICT OOF
        pred = model.predict(x_val)
        models.append(model) 
        
        # REPORT RESULTS
        try: 
            mse = mean_squared_error(np.nan_to_num(y_val), np.nan_to_num(pred))
        except: 
            mse = 0.0
        scores.append(mse)
        oof_preds[val_idx] = pred
        w_score = corr(np.nan_to_num(y_val), 
                       np.nan_to_num(pred.flatten()), 
                       np.array([asset_weight_dict[asset_id]] * len(y_val))
                       )
        print('#### FOLD %i OOF MSE %.3f | WCORR: %.3f' % (fold + 1, mse, w_score))

    orig_close = df['Close'].copy()
    df = df_proc
    df['Close'] = orig_close
    df['oof_preds'] = np.nan_to_num(oof_preds)
    print('\n\n' + ('-' * 80) + '\n' + 'Finished training %s. ' % asset_name_dict[asset_id])
    print('Results:')
    print('Model: r2_score: %s | pearsonr: %s | wcorr: %s ' % (
        r2_score(df['y'], df['oof_preds']), 
        pearsonr(df['y'], df['oof_preds'])[0], 
        corr(df['y'].values, df['oof_preds'].values, np.array([asset_weight_dict[asset_id]] * len(df['y'].values)))
        ))
    print('Predictions std: %s | Target std: %s' % (df['oof_preds'].std(), df['y'].std()))
    
    try: 
        plt.close()
    except: 
        pass   
    df2 = df.reset_index().set_index('date')
    
    fig = plt.figure(figsize = (12, 6))
    # fig, ax_left = plt.subplots(figsize = (12, 6))
    ax_left = fig.add_subplot(111)
    ax_left.set_facecolor('azure')    
    ax_right = ax_left.twinx()
    # 3ヶ月窓ごとの相関係数(Targetと予測値の)->赤線
    ax_left.plot(df2['y'].rolling(3 * 30 * 24 * 60).corr(df2['oof_preds']).iloc[::24 * 60], color = 'crimson', alpha=0.8, label = "Target WCorr")
    # 単純なClose値
    ax_right.plot(df2['Close'].iloc[::24 * 60], color = 'cornflowerblue', alpha=0.8, label = "%s Close" % asset_name_dict[asset_id])
    h1, l1 = ax_left.get_legend_handles_labels()
    h2, l2 = ax_right.get_legend_handles_labels()
    ax_left.set_ylabel(r'Target WCorr')
    ax_right.set_ylabel(r'Close')

    plt.grid()
    plt.legend(h1+h2, l1+l2, loc='lower right')
    plt.xlabel('Time')
    plt.title('3 month rolling pearsonr for %s' % (asset_name_dict[asset_id]))
    plt.show()

    fig = plt.figure(figsize = (12, 6))    
    feature_importances = []
    for model in (models):
        feature_importances.append(model.feature_importances_)
    feature_importances = np.array(feature_importances).mean(axis=0)

    importances = pd.DataFrame({'Feature': X.columns,
                                'Importance': np.round(feature_importances, 3)})

    importances = importances.head(15).sort_values('Importance', ascending = True).set_index('Feature')

    importances.plot.barh(color = 'deepskyblue', edgecolor = 'firebrick', legend = False)
    plt.title('Feature importances for %s' % (asset_name_dict[asset_id]))
    plt.xlabel('Importance');
    plt.show()
    
    return scores, oof_preds, models, y, scaler

In [ ]:
# Training
models, scores, targets, oof_preds, scalers = {}, {}, {}, {}, {}
for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    try:
        print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
        cur_scores, cur_oof_preds, cur_models, cur_targets, scaler = get_Xy_and_model_for_asset(asset_id)
        scores[asset_id], oof_preds[asset_id], models[asset_id], targets[asset_id], scalers[asset_id] = np.mean(cur_scores), cur_oof_preds, cur_models, cur_targets, scaler
    except: pass

# Calculate OOF MSE

In [ ]:
# COMPUTE OVERALL OOF MSE
print('Overall MEAN OOF MSE %s' % np.mean(list(scores.values())))

# SAVE OOF TO DISK 
y_pred, y_true, weights = [], [], []
for asset in oof_preds:
    df_oof = pd.DataFrame(dict(asset_id = asset, oof_preds=oof_preds[asset]))
    df_oof.to_csv(str(asset) + '_oof.csv',index=False)
    y_pred += oof_preds[asset].tolist()
    y_true += targets[asset].tolist() 
    weights += ([asset_weight_dict[asset]] * len(oof_preds[asset].tolist()))
    print('%s score: %s' % (asset_name_dict[asset], corr(np.nan_to_num(np.array(y_true).flatten()), np.nan_to_num(np.array(y_pred).flatten()), np.nan_to_num(np.array(weights).flatten()))))
    
print('Overall score %s' % corr(np.nan_to_num(np.array(y_true).flatten()), np.nan_to_num(np.array(y_pred).flatten()), np.nan_to_num(np.array(weights).flatten())))

# <span class="title-section w3-xxlarge" id="submit">Submit To Kaggle 🇰</span><hr>

In [ ]:
all_df_test = []

env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    # Feature Eng.
    df_test['date'] = pd.to_datetime(df_test['timestamp'], unit = 's')
    df_proc = get_features(df_test)
    df_proc['date'] = df_test['date'].copy()
    # df_proc['y'] = df['Target']
    # df_proc = df_proc.dropna(how="any")
    # X = df_proc.drop("y", axis=1)
    # y = df_proc["y"]
    groups = pd.factorize(df_proc['date'].dt.day.astype(str) + '_' + df_proc['date'].dt.month.astype(str) + '_' + df_proc['date'].dt.year.astype(str))[0]
    df_proc = df_proc.drop(columns = 'date')
    
    # df_procを1行ずつ処理
    # j-idx, row-data(1行分)
    for j , row in df_proc.iterrows():
        model = models[row['Asset_ID']]   # Asset_IDに対応するモデルLISTを呼び出す
        scaler = scalers[row['Asset_ID']] # Asset_IDに対応するSCALERを呼び出す
        # x_test = get_features(row)      # 説明変数を呼び込む
        x_test = row.to_frame().T 
        tmp_X = scaler.transform(x_test)
        x_test = pd.DataFrame(tmp_X, columns=x_test.columns)
        # inference
        y_preds = []
        for fold in range(FOLDS):
            y_pred = model[fold].predict(x_test)
            y_preds.append(y_pred)
        y_pred = np.array(y_preds).mean()

        row['row_id'] = int(df_pred.row_id.iloc[j])
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred

        # Print just one sample row to get a feeling of what it looks like
        if i == 0 and j == 0:
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        display(df_pred)
    all_df_test.append(df_test)

    # Send submissions/ ここでpredすることで、次のイテレーションに進める。
    env.predict(df_pred) 